In [1]:
!pip install SciExpeM-API 
!pip install SciExpeM-API --upgrade

     |████████████████████████████████| 130 kB 8.5 MB/s 
     |████████████████████████████████| 7.9 MB 42.5 MB/s 


In [2]:
from SciExpeM_API.SciExpeM import SciExpeM
my_sciexpem = SciExpeM(username='nikita.rozov', password='mdp2022_')

In [3]:
result_list = my_sciexpem.filterDatabase(model_name='Experiment', fileDOI='tmp_39904519421883596702')

print(result_list)
for result in result_list:
    print(result.__dict__)
    print("DOI:", result.fileDOI)
    print("Reactor:", result.reactor)
    print('Data columns', result._data_columns)
    for data in result._data_columns:
      print(data.__dict__)
    

[]


HTTP ERROR 403 -> {'detail': 'You do not have permission to perform this action.'}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
data = pd.read_excel("/content/drive/My Drive/MDP/POLIMI_1212_C1C3.xlsx")
data = data.drop(columns=["Unnamed: 0"])
data.head()

In [ ]:
[]

In [ ]:
x = data.drop(columns=['Experiment DOI','Exp SciExpeM ID','Chem Model','Error','d0L2','d1L2','d0Pe','d1Pe','shift'])
x_exp_type = pd.get_dummies(x['Experiment Type'], prefix='Experiment Type')
# dx = x.set_index('id').join(x_exp_type.set_index('image_id'))
y = data['Error']
x.head()

,Chem Model ID,Experiment Type,Reactor,Target,Fuels,Phi,Pressure (Bar),Temperature (K),Score
0,5,ignition delay measurement,shock tube,tau_OH(slope),"['H2', 'CO']","(0.5, 0.5)","(12.159, 12.159)","(1098.47, 1382.55)",0.812317
1,5,ignition delay measurement,shock tube,tau_OH(slope),['H2'],"(1.0, 1.0)","(1.216, 1.216)","(915.1, 1456.7)",0.820225
2,5,ignition delay measurement,shock tube,tau_OH(slope),['H2'],"(1.0, 1.0)","(111.5, 111.5)","(1154.0, 1291.0)",0.788749
3,5,ignition delay measurement,shock tube,tau_P(slope),['CH4'],"(2.0, 2.0)","(2.984, 4.012)","(1602.0, 1922.0)",0.807739
4,5,jet stirred reactor measurement,stirred reactor,CO2_x,"['CO', 'H2']","(0.1, 0.1)","(1.013, 1.013)","(849.15, 1199.15)",0.898507


In [ ]:
def splitter(x, num):
  first, second = x.replace('(','').replace(')','').split(", ")
  if num == 1:
    return first
  else:    
    return second
x['Phi1'] = x.Phi.apply(splitter,num = 1)
x['Pressure1'] = x['Pressure (Bar)'].apply(splitter,num = 1)
x['Temperature1'] = x["Temperature (K)"].apply(splitter,num = 1)
x['Phi2'] = x.Phi.apply(splitter,num = 2)
x['Pressure2'] = x['Pressure (Bar)'].apply(splitter,num = 2)
x['Temperature2'] = x["Temperature (K)"].apply(splitter,num = 2)
x = x.drop(columns=['Phi','Pressure (Bar)','Temperature (K)'])

x.head()

,Chem Model ID,Experiment Type,Reactor,Target,Fuels,Score,Phi1,Pressure1,Temperature1,Phi2,Pressure2,Temperature2
0,5,ignition delay measurement,shock tube,tau_OH(slope),"['H2', 'CO']",0.812317,0.5,12.159,1098.47,0.5,12.159,1382.55
1,5,ignition delay measurement,shock tube,tau_OH(slope),['H2'],0.820225,1.0,1.216,915.1,1.0,1.216,1456.7
2,5,ignition delay measurement,shock tube,tau_OH(slope),['H2'],0.788749,1.0,111.5,1154.0,1.0,111.5,1291.0
3,5,ignition delay measurement,shock tube,tau_P(slope),['CH4'],0.807739,2.0,2.984,1602.0,2.0,4.012,1922.0
4,5,jet stirred reactor measurement,stirred reactor,CO2_x,"['CO', 'H2']",0.898507,0.1,1.013,849.15,0.1,1.013,1199.15


In [ ]:
x_exp_type = pd.get_dummies(x['Fuels'], prefix='Experiment Type')


In [ ]:
from sklearn.linear_model import Ridge
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(x, test_size=0.2)
y_train, y_test = train_test_split(y, test_size=0.2)
clf = Ridge(alpha=1.0)
clf.fit(x_train, y_train)

ValueError: ignored